In [1]:
from pipeline.preprocessing import data_wrangling, one_hot_encoding, feature_bandpower_struct, \
    feature_mfcc, feature_chromagram, feature_melspectrogram
from pipeline.dataloader import PhonocardiogramAudioDataset, PhonocardiogramByIDDatasetOnlyResult, \
    PhonocardiogramAugmentationTSV
from tqdm import tqdm
from pipeline.utils import compose_feature_label, audio_random_windowing
import pandas as pd
import librosa
import numpy as np
import os

In [2]:
from pathlib import Path
from torch.utils.data import DataLoader
import torch
import re

In [3]:
file = Path(".") / "assets" / "the-circor-digiscope-phonocardiogram-dataset-1.0.3"
# Training On CSV data
original_data = pd.read_csv(str(file  / "training_data.csv"))
    
model_df = data_wrangling(original_data)
X_CSV = one_hot_encoding(model_df, [
    'Murmur', 
    'Systolic murmur quality', 
    'Systolic murmur pitch',
    'Systolic murmur grading', 
    'Systolic murmur shape', 
    'Systolic murmur timing',
    'Diastolic murmur quality', 
    'Diastolic murmur pitch',
    'Diastolic murmur grading', 
    'Diastolic murmur shape', 
    'Diastolic murmur timing',
])
y_CSV = model_df['Outcome']

c:\Python Work\Senior Design Local\HeartDiseasePrediction\pipeline\preprocessing.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Age"].fillna(
c:\Python Work\Senior Design Local\HeartDiseasePrediction\pipeline\preprocessing.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["Outcom

In [4]:
# Training on actual patient audio files
segmentation_table = PhonocardiogramAugmentationTSV(file / "training_data")

def augmentation(data, sr=4000, window_length_hz=200, window_len_sec =5.):
    # This augmentation WILL conflict with new feature of frequency based extraction. ->
    x = data
    # x = energy_band_augmentation_random_win(x, sr=sr, window_hz_length=window_length_hz)
    # x = np.fft.ifft(x).real
        
    x = audio_random_windowing(x, window_len_sec)
    return x

In [5]:
def feature_csv(file):
    match = re.match(r'(\d+)_(AV|TV|MV|PV|Phc)', os.path.basename(file))
    key = int(match.group(1))
    record = X_CSV.loc[original_data["Patient ID"] == key].to_numpy()[0]
    return record

def compose_with_csv(file, audio_extracted_features_label):
    feature, y = audio_extracted_features_label
    csv_feat = feature_csv(file)
    return np.concatenate([feature, csv_feat], axis=0), y

In [6]:
features_fn = [
    feature_mfcc,
    feature_chromagram, 
    feature_melspectrogram,
    feature_bandpower_struct(4000,200,0.7),
]

In [7]:
lookup = PhonocardiogramByIDDatasetOnlyResult(str(file / "training_data.csv"))
if features_fn == []:
    dset = PhonocardiogramAudioDataset(
        file / "training_data",
        ".wav",
        "*", # Everything
        transform=lambda f: (augmentation(librosa.load(f)[0],4000,300,3.),int(lookup[f])),
        balancing=True,
        csvfile=str(file / "training_data.csv"),
        shuffle=True
    )
else:
    dset = PhonocardiogramAudioDataset(
        file / "training_data",
        ".wav",
        "*", # Everything
        transform=lambda f : compose_with_csv(f, compose_feature_label(
            f,
            lookup, 
            features_fn,
            lambda ary_data : augmentation(ary_data,4000,300,3.))
        ),  
        balancing=True,
        csvfile=str(file / "training_data.csv"),
        shuffle=True
    )

loader = DataLoader(
    dset, 
    batch_size=1,
    shuffle=True
)
X = []
y = []

for resample in range(BATCHING := 1):
    for i in tqdm(loader): # very slow 
        X_i,y_i = i
        X.append(X_i)
        y.append(y_i)

# Creating 1 large matrix to train with classical models
X = torch.cat(X, dim=0)
y = torch.cat(y, dim=0)

  0%|          | 8/3060 [00:01<05:21,  9.49it/s]C:\Users\songy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
100%|██████████| 3060/3060 [00:42<00:00, 72.41it/s]


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



def get_cnn_model(input_shape):
    cnn_model = Sequential()

    # Convolutional layer
    cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.2))

    # Another convolutional layer
    cnn_model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.2))

    # # Third convolutional layer
    # cnn_model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    # cnn_model.add(BatchNormalization())
    # cnn_model.add(MaxPooling1D(pool_size=2))
    # cnn_model.add(Dropout(0.3))

    # Flattening followed by dense layers
    cnn_model.add(Flatten())
    cnn_model.add(Dense(256, activation='relu'))
    cnn_model.add(Dropout(0.5))
    cnn_model.add(Dense(1, activation='sigmoid'))  
    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    cnn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

    return cnn_model, early_stop

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def get_convrnn_model(input_shape):
    convrnn_model = Sequential()

    # Convolutional layer
    convrnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    convrnn_model.add(BatchNormalization())
    convrnn_model.add(MaxPooling1D(pool_size=2))
    convrnn_model.add(Dropout(0.2))

    # Another convolutional layer
    convrnn_model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    convrnn_model.add(BatchNormalization())
    convrnn_model.add(MaxPooling1D(pool_size=2))
    convrnn_model.add(Dropout(0.2))

    # Recurrent layer (LSTM)
    convrnn_model.add(LSTM(64, return_sequences=True))
    convrnn_model.add(LSTM(32, return_sequences=True))

    # Flattening followed by dense layers
    convrnn_model.add(Flatten())
    convrnn_model.add(Dense(256, activation='relu'))
    convrnn_model.add(Dropout(0.5))
    convrnn_model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    convrnn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    return convrnn_model, early_stop

In [10]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
def cnn_train(X,y):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    cnn, early_stop = get_cnn_model((X_train.shape[1],1))
    # cnn.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), verbose=1, callbacks=[early_stop])
    cnn.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=1)

    probabilities = cnn.predict(X_test)
    threshold = 0.5
    y_pred = (probabilities >= threshold).astype(int)

    # y_pred = np.round(y_pred).astype(int)  # Convert probabilities to binary labels

    acc = metrics.accuracy_score(y_test, y_pred)
    fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {acc}")
    print(f"Auc: {auc}")
    print(f"F1 Score: {f1}")
    acc = round(acc * 100, 2)
    auc = round(auc * 100, 2)
    f1 = round(f1 * 100, 2)
    return acc, auc, f1

def convrnn_train(X,y):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    convrnn, early_stop = get_convrnn_model((X_train.shape[1],1))
    # convrnn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[early_stop])
    convrnn.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=1)
    probabilities = convrnn.predict(X_test)
    threshold = 0.5
    y_pred = (probabilities >= threshold).astype(int)

    # y_pred = np.round(y_pred).astype(int)  # Convert probabilities to binary labels

    acc = metrics.accuracy_score(y_test, y_pred)
    fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {acc}")
    print(f"Auc: {auc}")
    print(f"F1 Score: {f1}")
    acc = round(acc * 100, 2)
    auc = round(auc * 100, 2)
    f1 = round(f1 * 100, 2)
    return acc, auc, f1

In [11]:
# Training Pipeline
cnn_train(X,y)

Epoch 1/30


C:\Users\songy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5147 - loss: 1.6742 - val_accuracy: 0.5310 - val_loss: 0.6852
Epoch 2/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5480 - loss: 0.7241 - val_accuracy: 0.4788 - val_loss: 0.7261
Epoch 3/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5321 - loss: 0.7297 - val_accuracy: 0.4739 - val_loss: 0.7257
Epoch 4/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5530 - loss: 0.6866 - val_accuracy: 0.4837 - val_loss: 0.7209
Epoch 5/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5809 - loss: 0.6789 - val_accuracy: 0.4788 - val_loss: 0.7849
Epoch 6/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5993 - loss: 0.6547 - val_accuracy: 0.4820 - val_loss: 0.7297
Epoch 7/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6317 - loss: 0.6416 - val_accuracy: 0.5033 - val_loss: 0.7030
Epoch 8/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6199 - loss: 0.6538 - val_accuracy: 0.4788 - val_loss: 0.

(56.54, 56.2, 65.63)

In [12]:
## 
convrnn_train(X,y)

Epoch 1/30


C:\Users\songy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.5230 - loss: 0.6967 - val_accuracy: 0.4788 - val_loss: 0.7699
Epoch 2/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5781 - loss: 0.6622 - val_accuracy: 0.4788 - val_loss: 0.7618
Epoch 3/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6074 - loss: 0.6395 - val_accuracy: 0.4788 - val_loss: 0.9615
Epoch 4/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6004 - loss: 0.6475 - val_accuracy: 0.4788 - val_loss: 0.7977
Epoch 5/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.6423 - loss: 0.6261 - val_accuracy: 0.4771 - val_loss: 0.7347
Epoch 6/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6296 - loss: 0.6259 - val_accuracy: 0.4788 - val_loss: 0.8506
Epoch 7/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.6312 - loss: 0.6261 - val_accuracy: 0.4788 - val_loss: 0.7859
Epoch 8/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6173 - loss: 0.6384 - val_accuracy: 0.4788 - val_loss: 1.

(50.98, 50.35, 67.11)

In [33]:
## feature_melspectrogram
convrnn_train(X,y)

Epoch 1/100


C:\Users\songy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5935 - loss: 0.6644 - val_accuracy: 0.5212 - val_loss: 0.6844
Epoch 2/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6506 - loss: 0.6356 - val_accuracy: 0.5098 - val_loss: 0.6852
Epoch 3/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6426 - loss: 0.6259 - val_accuracy: 0.5180 - val_loss: 0.6856
Epoch 4/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6461 - loss: 0.6303 - val_accuracy: 0.5196 - val_loss: 0.6832
Epoch 5/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6303 - loss: 0.6322 - val_accuracy: 0.5212 - val_loss: 0.6864
Epoch 6/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6211 - loss: 0.6287 - val_accuracy: 0.6160 - val_loss: 0.6695
Epoch 7/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6292 - loss: 0.6237 - val_accuracy: 0.6716 - val_loss: 0.6302
Epoch 8/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6465 - loss: 0.6253 - val_accuracy: 0.6748 - val_loss: 0.

In [36]:
##   feature_chromagram, 
acc, auc, f1 = convrnn_train(X,y)

Epoch 1/100


C:\Users\songy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.6088 - loss: 0.6593 - val_accuracy: 0.6095 - val_loss: 0.6835
Epoch 2/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6399 - loss: 0.6315 - val_accuracy: 0.5670 - val_loss: 0.6786
Epoch 3/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6358 - loss: 0.6257 - val_accuracy: 0.6111 - val_loss: 0.6703
Epoch 4/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6467 - loss: 0.6262 - val_accuracy: 0.6536 - val_loss: 0.6548
Epoch 5/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6193 - loss: 0.6298 - val_accuracy: 0.6585 - val_loss: 0.6410
Epoch 6/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6427 - loss: 0.6262 - val_accuracy: 0.6699 - val_loss: 0.6263
Epoch 7/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6447 - loss: 0.6245 - val_accuracy: 0.6683 - val_loss: 0.6064
Epoch 8/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6602 - loss: 0.6175 - val_accuracy: 0.6683 - val_loss: 0.

In [37]:
## feature_mfcc
convrnn_train(X,y)

Epoch 1/100


C:\Users\songy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5980 - loss: 0.6627 - val_accuracy: 0.5196 - val_loss: 0.6866
Epoch 2/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6236 - loss: 0.6447 - val_accuracy: 0.5229 - val_loss: 0.6796
Epoch 3/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6288 - loss: 0.6351 - val_accuracy: 0.5310 - val_loss: 0.6766
Epoch 4/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6314 - loss: 0.6225 - val_accuracy: 0.6046 - val_loss: 0.6695
Epoch 5/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6533 - loss: 0.6224 - val_accuracy: 0.6193 - val_loss: 0.6644
Epoch 6/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6337 - loss: 0.6346 - val_accuracy: 0.6634 - val_loss: 0.6289
Epoch 7/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6393 - loss: 0.6250 - val_accuracy: 0.6683 - val_loss: 0.6005
Epoch 8/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6498 - loss: 0.6209 - val_accuracy: 0.6716 - val_loss: 0.

(64.38, 64.87, 50.0)